# 数据载入模块
目标：给定数据集所在路径，返回整个文件经过处理后而成的id索引以及标签

In [2]:
import numpy as np
import pandas as pd
import torch
from make_vocab import lst_gram, n_gram

['#ke', 'key', 'eyb', 'ybo', 'boa', 'oar', 'ard', 'rdk', 'dke', 'key', 'eyb', 'ybo', 'boa', 'oar', 'ard', 'rd#']
['#tw', 'two', 'wo#', '#yo', 'you', 'oun', 'ung', 'ng#', '#ch', 'chi', 'hil', 'ild', 'ldr', 'dre', 'ren', 'en#', '#in', 'in#', '#bl', 'blu', 'lue', 'ue#', '#je', 'jer', 'ers', 'rse', 'sey', 'eys', 'ys#']
{'la.', '4.8', 'jcp', '#gr', '5.7', 'tph', 'thq', 'iok', 'f=#', 'eat', 'e-d', '1%#', 'ymb', 'ek#', 'st.', '421', 'tro', 's.g', 'ofi', 'ub#', 'wsm', 'cod', 'rs.', 'hwa', 'rik', 'ait', 'ya#', 'tly', '0gh', 'rst', '990', 'ut]', 'act', '#ft', 'eyw', 'fet', '255', 'gou', 'ffe', 'mbu', 'orm', 'uu#', 'avi', '#97', 'ems', 'lil', 'ke.', 'qat', 'isf', 'itc', 'l-c', '81#', 'o-i', '961', '5-i', 'yn#', 'fer', 'jup', 'eys', 'vow', '#kh', '#vo', 'km.', 'iff', 'rav', 'bis', 'oke', 'bie', '16)', 'uny', '26t', 'rpr', 'nxt', 'gmt', '6.4', 'way', 's-l', 'lud', '#he', 'ea#', 'dj.', '.9%', '#hw', 'mor', 're-', 'sti', 'ypl', 'bbo', '#ar', 'ixe', 'nwr', '#pc', '-fu', 's25', '20-', 'kma', 'fma', 'bn

vocab_mrpc.txt文件，建立切片到索引，索引到切片间的双向关系

In [4]:
def load_vocab():
    vocab = open('./vocab_mrpc.txt',encoding='utf-8').readlines()
    slice2idx = {}
    idx2slice = {}
    cnt = 0
    for char in vocab:
        char = char.strip('\n')
        slice2idx[char]=cnt
        idx2slice[cnt]=char
        cnt+=1
    return slice2idx, idx2slice

## 函数应用举例

In [5]:
slice2idx, idx2slice=load_vocab()
text='do you like what you see'
text_ids=[]
id_list=[234,45,10,656,77,2091,2009,9872,254,555,10]

text_gram=lst_gram(text,n=3)
print("将'do you like what you see'文本转化为id序列：",end="")
for char in text_gram:
    print(slice2idx[char],end=' ')
    text_ids.append(slice2idx[char])
    
print("\nid_list转化为字符切片为：",end="")
for idx in id_list:
    print(idx2slice[idx],end=" ")

将'do you like what you see'文本转化为id序列：1472 995 6106 5822 2102 4046 5635 7227 8134 2575 5306 7403 7179 6106 5822 2102 8544 1337 2725 
id_list转化为字符切片为：od. #97 f=# d.c way (ny xid n@d iab aah f=# 

## 数据填充，截取，保证送入模型的数据格式规整

In [7]:
def padding(text, maxlen=70):
    pad_text=[]
    for sentence in text:
        pad_sentence = np.zeros(maxlen).astype("int64")
        cnt=0
        for index in sentence:
            pad_sentence[cnt]=index
            cnt+=1
            if cnt==maxlen:
                break
        pad_text.append(pad_sentence.tolist())
    return pad_text

In [8]:
# 函数应用举例
text=[[16,120,48,512],[234,45,10,656,77,2091],
     list(range(80))]
pad_text=padding(text)
print(pad_text)

[[16, 120, 48, 512, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [234, 45, 10, 656, 77, 2091, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]]


# 整合上两函数，给定批量的文本对，返回他们的索引id
## 考虑未出现过的单词与截断

In [9]:
def char_index(text_a, text_b):
    slice2idx, idx2slice=load_vocab()
    a_list, b_list=[], []
    # 对文件中的每一行
    for a_sentence, b_sentence in zip(text_a, text_b):
        a, b = [], []
        # 对每一行中的每一个切片
        for slice in lst_gram(a_sentence):
            if slice in slice2idx.keys():
                a.append(slice2idx[slice])
            else:
                a.append(1)  #没被切片的记作UNK
                
        for slice in lst_gram(b_sentence):
            if slice in slice2idx.keys():
                b.append(slice2idx[slice])
            else:
                b.append(1)
                
        a_list.append(a)
        b_list.append(b)
        
    a_list=padding(a_list)
    b_list=padding(b_list)
    
    return a_list, b_list


In [10]:
ta=['Families waiting in line at an amusement park','##)']
tb=['People are riding bikes.','#&^']
a_list,b_list=char_index(ta,tb)

print(a_list)
print(b_list)

[[5038, 5818, 2285, 8143, 1044, 9567, 3022, 5251, 3461, 4020, 27, 839, 5372, 984, 1429, 2116, 6698, 4046, 2565, 3955, 7954, 3783, 7179, 4399, 7784, 362, 8624, 4700, 6055, 4841, 5050, 6647, 7239, 8818, 8422, 1597, 6313, 2212, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[228, 664, 1886, 7887, 7306, 9890, 90, 9115, 9627, 8861, 4140, 6208, 6861, 984, 1429, 7890, 6521, 7227, 402, 1507, 6549, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
def load_char_data(filename):
    df = pd.read_csv(filename,sep='\t')
    text_a=df['#1 string'].values
    text_b=df['#2 string'].values
    label=df['quality'].values
    a_index, b_index = char_index(text_a, text_b)
    return np.array(a_index), np.array(b_index), np.array(label)

In [12]:
a_index,b_index,label=load_char_data('./MRPC/test_data.csv')

In [13]:
len(a_index)

1725

In [14]:
a_index[:7]

array([[   93,  4966,  8428,  2001,  5052,  8773,   908,  5209,  3170,
         4983,  8142,  2083,  5478,  2509,   235,  1887,  5372,   984,
         1429,  8917,  6850,  7220,  2802,  7604,  5194,  8633,   741,
         3024,  7227,  8134,  4790,  8645,  6160,  1700,  6891,  4223,
         8633,  4399,  3935,  2106,  6151,  9546,  1731,  8465,    90,
         9115,  9247,  3118,  3023,  8644,  2840,  9651,  8773,   908,
         5209,  3170,  4983,  2353,  5957,  1153,  1479,   890,   754,
         5643,  5039,  6745,  8917,  6850,  7220,  2802],
       [ 8644,  2840,  9651,   329,  4714,  3989,  4502,   705,  3077,
         2455,  6216,  8124,  4438,  1516,  7900,  9742,  1264,  3053,
         3172,   570,  4810,   295,  7085,  8671,  4706,  9560,   241,
          676,   937,  2239,  4624,  3377,  3176,  8644,  2840,  3126,
         5767,  7987,   810,   461,  6035,  6549,  2239,  9391,  9546,
         7116,  5251,  9147,  4483,  8710,   690,  2565,  3955,  3809,
         5059,  518

In [15]:
label[0:7]

array([1, 1, 1, 0, 0, 1, 0], dtype=int64)